# Put title and about section ML menciona los objetivos y así

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split


In [ ]:
# Load the preprocessed ESG and Holdings data
esg_2024= pd.read_parquet('esg_2024')

In [ ]:
# Show the data
esg_2024.sample(3)

,Environmental,Social,Governance,Climate_change,ESG_any,Region_Africa,Region_Asia,Region_Europe,Region_Latin America,Region_Middle East,...,Industry_Financials,Industry_Health Care,Industry_Industrials,Industry_Real Estate,Industry_Technology,Industry_Telecommunications,Industry_Utilities,Ownership_log_scaled,Market_Value_USD_log_scaled,Portfolio_Weight_scaled
6450,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0.273454,0.721235,0.001291
5236,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0.336456,0.651538,0.000245
6161,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0.206777,0.841667,0.022860


In [ ]:
# Checking if there are missing values
esg_2024.isnull().any().any()   

False

cambia a collab mañana, busca los gridsearch de modelos, busca el AutoML y  pregúntale al chat la métrica adecuada para ese autoML

In [ ]:
pd.set_option('display.max_columns', None) # From now on, all columns will be shown when displaying a dataframe

In [10]:
esg_2024.columns

Index(['Environmental', 'Social', 'Governance', 'Climate_change', 'ESG_any',
       'Region_Africa', 'Region_Asia', 'Region_Europe', 'Region_Latin America',
       'Region_Middle East', 'Region_North America', 'Region_Oceania',
       'Industry_Basic Materials', 'Industry_Consumer Discretionary',
       'Industry_Consumer Staples', 'Industry_Energy', 'Industry_Financials',
       'Industry_Health Care', 'Industry_Industrials', 'Industry_Real Estate',
       'Industry_Technology', 'Industry_Telecommunications',
       'Industry_Utilities', 'Ownership_log_scaled',
       'Market_Value_USD_log_scaled', 'Portfolio_Weight_scaled'],
      dtype='object')

# PULIR First objective:  Find explanatory variables for ESG_any

**Why Classic Logistic Regression is skipped**

High-dimensional categorical variables: Many industry and region dummies create near-collinear features, leading to singular matrices and unstable coefficient estimates.

Extreme class imbalance (≈89/11): Logistic regression struggles with quasi-complete separation; MLE may fail to converge.

Sparse, high-variance interactions: Some combinations of region/industry predict ESG_any almost perfectly, breaking logistic assumptions.

Interpretability alternatives exist: Tree-based models like XGBoost allow feature importance and SHAP for explanation, without requiring linearity or dropping categories.

In short, linear/logit models would produce misleading coefficients and potentially infinite parameter estimates. Gradient boosting is robust, handles imbalance, and preserves all variables for ML patterns.

In [ ]:
#  H2O AutoML
import h2o
from h2o.automl import H2OAutoML

h2o.init(max_mem_size="4G")

hf = h2o.H2OFrame(esg_2024)
hf['ESG_any'] = hf['ESG_any'].asfactor()  # target as categorical

train, test = hf.split_frame(ratios=[0.8], seed=42)

aml = H2OAutoML(max_models=20,
                seed=42,
                balance_classes=True,  # very important due to 11% positives
                sort_metric="AUC")    # or "pr_auc" for precision-recall

aml.train(y='ESG_any', training_frame=train)

lb = aml.leaderboard
print(lb)
